# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 81.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=1c164e89e9490a0a8d6b1edfb603099a1017c8a0a643d3596631a9fb5054bac1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.metrics.distance import edit_distance
import random
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [ ]:
with open('litw-win.txt', encoding = 'cp1251') as f:
    words1 = [row.split()[-1] for row in f]
text_words = list(text.split())
for i in range(len(text_words)):
    if text_words[i] not in words1:
        distances = {j : edit_distance(text_words[i], j) for j in words1}
        sorted_distances = sorted(distances.items(), key = lambda x: x[1])
        text_words[i] = sorted_distances[0][0]
print(' '.join(text_words))


с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
text2 = 'Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words'

In [ ]:
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()
text2_words = [word for word in word_tokenize(str(text2)) if word.isalpha()]

In [ ]:
for i in text2_words:
    print(stemmer.stem(i))

счита
слов
из
файл
и
запиш
их
в
список
words
в
зада
предложен
исправьт
все
опечатк
замен
слов
с
опечатк
на
ближайш
в
смысл
расстоян
левенштейн
к
ним
слов
из
списк
words
счита
что
в
слов
ест
опечатк
есл
дан
слов
не
содерж
в
списк
words


In [ ]:
for i in text2_words:
    print(morph.parse(i)[0].normalized.word)

считать
слово
из
файл
и
записать
они
в
список
words
в
задать
предложение
исправить
всё
опечатка
заменить
слово
с
опечатка
на
близкий
в
смысл
расстояние
левенштейн
к
они
слово
из
список
words
считать
что
в
слово
есть
опечатка
если
данный
слово
не
содержаться
в
список
words


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
import nltk
cv = CountVectorizer()
cv.fit_transform(nltk.sent_tokenize(text2)).toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
data = pd.read_csv('preprocessed_descriptions.csv')
data

,name,preprocessed_descriptions
0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,i can t believe it s spinach,these were so go it surprised even me
3,italian gut busters,my sisterinlaw made these for us at a family g...
4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...
29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,zydeco soup,this is a delicious soup that i originally fou...


In [ ]:
all_words = []
for i in data['preprocessed_descriptions']:
    all_words.extend([word for word in word_tokenize(str(i)) if word.isalpha()])
words = set(all_words)
print(words)

{'julienned', 'mandell', 'fame', 'carefully', 'bakingpowderleavened', 'floridanative', 'talent', 'corona', 'vegetablesthis', 'urgehot', 'nims', 'cutest', 'mucha', 'cocker', 'grumpy', 'rotisiree', 'regional', 'tony', 'leaflet', 'jumpstart', 'naan', 'originality', 'croatians', 'charge', 'graeme', 'warrington', 'noncooks', 'german', 'saunders', 'goblets', 'triscuts', 'embark', 'doesi', 'pretend', 'safety', 'dishpoints', 'amarillo', 'bobbys', 'cutting', 'homeliest', 'recipeperfect', 'vol', 'zaarenjoy', 'ryan', 'basement', 'expected', 'greatfalls', 'salsa', 'surprises', 'newfie', 'wboiled', 'heathers', 'changeable', 'sprtiz', 'greenleebellsouthnet', 'needsgood', 'hotel', 'thx', 'substantially', 'permeate', 'familly', 'unsolicited', 'vicious', 'rude', 'gerrys', 'squeezed', 'conventional', 'jell', 'underbaked', 'glutenwheat', 'refrigerateafter', 'deceiving', 'averse', 'cookielike', 'chestnutlike', 'dusty', 'success', 'dishyou', 'variables', 'sug', 'tina', 'freshsqueezed', 'fiveseed', 'laughed

In [ ]:
len(words)

30743

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
for i in range(5):
    w1, w2 = random.sample(list(words), 2)
    print(f'Слова: {w1}, {w2}. Расстояние редактирования: {edit_distance(w1, w2)}\n')

Слова: everday, raspstyle. Расстояние редактирования: 8

Слова: awsome, treasured. Расстояние редактирования: 7

Слова: formpan, stacking. Расстояние редактирования: 7

Слова: handi, garden. Расстояние редактирования: 4

Слова: besa, otheri. Расстояние редактирования: 5



1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def dist(word, k):
    dists = {i : edit_distance(word, i) for i in words}
    sorted_dists = sorted(dists.items(), key = lambda x: x[1])
    return sorted_dists[:k]

In [ ]:
dist('database', 5)

[('database', 0),
 ('oatbased', 3),
 ('patatas', 3),
 ('bahamas', 4),
 ('tabasco', 4)]

In [ ]:
dist('description', 7)

[('description', 0),
 ('descriptions', 1),
 ('desciption', 1),
 ('descriptive', 2),
 ('prescription', 2),
 ('subscription', 3),
 ('prescripton', 3)]

### Стемминг, лемматизация

In [ ]:
import nltk
nltk.download('wordnet')
  

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
df_words = pd.DataFrame(words, columns = ['word'])
df_words['stemmed_word'] = df_words.apply(lambda x: stemmer.stem(x['word']), axis = 1)
df_words['normalized_word'] = df_words.apply(lambda x: lemmatizer.lemmatize(x['word']), axis = 1)
df_words = df_words.set_index('word')
df_words

,stemmed_word,normalized_word
word,,
julienned,julien,julienned
mandell,mandel,mandell
fame,fame,fame
carefully,care,carefully
bakingpowderleavened,bakingpowderleaven,bakingpowderleavened
...,...,...
cerveza,cerveza,cerveza
partook,partook,partook
knocked,knock,knocked


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
words_no_stop = [word for word in all_words if word not in stop_words]
print(f'Доля стоп слов составляла: {(len(all_words) - len(words_no_stop)) / len(all_words) * 100}')

Доля стоп слов составляла: 46.10722870409009


In [ ]:
for k, v in nltk.FreqDist(all_words).most_common(10):
    print(f'{k} - {v}')

the - 40072
a - 34951
and - 30245
this - 26859
i - 24836
to - 23471
is - 20285
it - 19756
of - 18364
for - 15939


In [ ]:
for k, v in nltk.FreqDist(words_no_stop).most_common(10):
    print(f'{k} - {v}')

recipe - 14871
make - 6326
time - 5137
use - 4620
great - 4430
like - 4167
easy - 4152
one - 3872
made - 3810
good - 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
data3 = data.sample(5)
data3

,name,preprocessed_descriptions
27227,thai chicken wraps,a lightened restaurant favorite so you get a t...
19259,open omelette,a great breakfast treat simple to make and wit...
2712,beefy burger dogs,a different kind of hot dog
2103,banana bran pancakes diabetic,these are a perfect sunday morning breakfast w...
29348,whole wheat biscuits with flax oamc option also,i finally perfected my baking powder biscuits ...


In [ ]:
tv = TfidfVectorizer()
for i in data3['preprocessed_descriptions']:
    print(tv.fit_transform(sent_tokenize(i)).toarray(), '\n')

[[0.12700013 0.12700013 0.25400025 0.12700013 0.12700013 0.25400025
  0.12700013 0.12700013 0.25400025 0.12700013 0.25400025 0.12700013
  0.12700013 0.25400025 0.12700013 0.12700013 0.12700013 0.12700013
  0.12700013 0.12700013 0.12700013 0.12700013 0.12700013 0.12700013
  0.12700013 0.12700013 0.25400025 0.25400025 0.12700013 0.25400025
  0.12700013 0.12700013 0.12700013 0.12700013 0.25400025]] 

[[0.21821789 0.21821789 0.21821789 0.21821789 0.21821789 0.21821789
  0.21821789 0.21821789 0.21821789 0.21821789 0.21821789 0.21821789
  0.21821789 0.21821789 0.21821789 0.21821789 0.21821789 0.21821789
  0.21821789 0.21821789 0.21821789]] 

[[0.4472136 0.4472136 0.4472136 0.4472136 0.4472136]] 

[[0.10259784 0.10259784 0.20519567 0.10259784 0.10259784 0.10259784
  0.10259784 0.10259784 0.10259784 0.20519567 0.10259784 0.20519567
  0.10259784 0.10259784 0.10259784 0.10259784 0.10259784 0.10259784
  0.10259784 0.10259784 0.10259784 0.10259784 0.30779351 0.20519567
  0.10259784 0.10259784 0.10

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
from scipy.spatial.distance import cosine
data3['preprocessed_descriptions'] = data3['preprocessed_descriptions'].astype(str)
tv = TfidfVectorizer()
vectors = tv.fit_transform(data3['preprocessed_descriptions']).toarray()
df3_2 = pd.DataFrame(index = data3['name'], columns = data3['name'])
for i, r1 in enumerate(data3['name']):
    for j, r2 in enumerate(data3['name']):
        df3_2.at[r1, r2] = 1 - cosine(vectors[i], vectors[j])
df3_2

name,thai chicken wraps,open omelette,beefy burger dogs,banana bran pancakes diabetic,whole wheat biscuits with flax oamc option also
name,,,,,
thai chicken wraps,1,0.041549,0.0,0.15856,0.204589
open omelette,0.041549,1,0.037284,0.128669,0.090883
beefy burger dogs,0.0,0.037284,1,0.058646,0.009234
banana bran pancakes diabetic,0.15856,0.128669,0.058646,1,0.26579
whole wheat biscuits with flax oamc option also,0.204589,0.090883,0.009234,0.26579,1


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [ ]:
res = df3_2.where(df3_2 != 1).max().max()
res

0.26578991902308013